In [2]:
%cd ..

/home/smehta/Projects/genea/CLIP


In [3]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0.9927   0.00412  0.002968]]


In [4]:
text

tensor([[49406,   320, 22697, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [49406,   320,  1929, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0, 

In [5]:
import pandas as pd

In [11]:
a = pd.read_pickle('data/genea2023/trn/main-agent/data2vec-audio/trn_2023_v0_000_main-agent.muted.pkl')
t = pd.read_pickle('data/genea2023/trn/main-agent/data2vec-text/trn_2023_v0_000_main-agent.pkl')
m = pd.read_pickle('data/genea2023/trn/main-agent/diffusion_genea23_sm0_0_30fps/trn_2023_v0_000_main-agent.bvh_expmap_30fps.pkl')


In [12]:
a.shape, t.shape, m.shape

((1950, 768), (1906, 768), (1950, 60))